**Poin penilaian :**

* Tingkat kesesuaian program
* Kesesuaian step preprocessing dan pemodelan
* Pemahaman materi
* Kerapian kode program
* Tidak ada error

# Content Based Filtering : Movie Recommender System

**Import library**

In [0]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

**Gunakan dataset berikut**

In [0]:
!wget -O moviedataset.zip http://files.grouplens.org/datasets/movielens/ml-latest.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-05-01 16:26:06--  http://files.grouplens.org/datasets/movielens/ml-latest.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277113433 (264M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 264.28M  72.9MB/s    in 3.9s    

2020-05-01 16:26:10 (68.4 MB/s) - ‘moviedataset.zip’ saved [277113433/277113433]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: tags.csv                
  inflating: genome-tags.csv         
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: genome-scores.csv       
  inflating: movies.csv              


Sekarang mari kita baca setiap file ke dalam Dataframe:

In [0]:
movies_df = pd.read_csv ('movies.csv')
ratings_df = pd.read_csv ('ratings.csv')
movies_df.head ()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


##### **Kegiatan 1 : 5 poin**
Hapus tahun dari kolom `title` dengan menggunakan fungsi `replace` pandas dan menyimpannya di kolom `year` yang baru.

In [0]:
# TULIS KODE ANDA DISINI

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


##### **Kegiatan 2 : 5 poin**

Ubah kolom **Genres** menjadi sebuah **list of Genre**.

In [0]:
# TULIS KODE ANDA DISINI

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


##### **Kegiatan 3 : 15 poin**

Ubah kolom genres kedalam bentuk **One Hot Encoding** kemudian simpan dalam dataframe baru.

Karena genre dalam format list tidak optimal untuk teknik sistem rekomendasi berbasis konten, kita akan menggunakan teknik **One Hot Encoding** untuk mengubah daftar genre menjadi vektor di mana setiap kolom sesuai dengan satu kemungkinan nilai fitur. Pengkodean ini diperlukan untuk memasukkan data kategori. Dalam hal ini, kita akan menyimpan setiap genre berbeda dalam kolom yang berisi 1 atau 0. 1 menunjukkan bahwa film memiliki genre tersebut dan 0 menunjukkan bahwa itu tidak.

In [0]:
# TULIS KODE ANDA DISINI

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Selanjutnya, mari kita lihat pada dataframe ratings.

In [0]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Setiap baris dalam dataframe ratings memiliki id pengguna yang terkait dengan setidaknya satu film, peringkat, dan timestamp yang ditampilkan ketika mereka memeriksanya. Kita tidak akan memerlukan kolom timestamp, jadi mari letakkan untuk menghemat memori.

In [0]:
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<a id="ref3"></a>
## **Content-Based recommendation system**

Sekarang, mari kita lihat bagaimana menerapkan __Content-Based__ atau __Item-item Recommendation systems__. Teknik ini mencoba untuk mencari tahu apa aspek favorit pengguna dari suatu item dan kemudian merekomendasikan item yang menyajikan aspek-aspek tersebut. Dalam kasus kami, kami akan mencoba mencari tahu genre input favorit dari film dan peringkat yang diberikan.

Mari kita mulai dengan membuat pengguna input untuk merekomendasikan film ke:

Perhatikan: Untuk menambahkan lebih banyak film, cukup tambahkan jumlah elemen dalam __userInput__. Jangan ragu untuk menambahkan lebih banyak! Pastikan untuk menuliskannya dengan huruf kapital dan jika film dimulai dengan "The", seperti "The Matrix" kemudian tulis seperti ini: 'Matrix, The'.

##### **Kegiatan 4 : 15 poin**

Buat user input berupa list of dictionary yang terdiri dari 6 dictionary yang berisi judul film dan rating yang berbeda.

Ubah list of dictionary tersebut kedalam dataframe `inputMovies` dan tampilkan isi dataframe tersebut.

# **NOTE : Pastikan Tidak Ada Isi List of Dictionary Yang Sama Persis Dengan Praktikan Lain**

In [0]:
# TULIS KODE ANDA DISINI

"""HANYA CONTOH OUTPUT"""

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


### **Add movieId to input user**

Dengan input yang lengkap, mari kita ekstrak ID film input dari bingkai data film dan menambahkannya ke dalamnya.

Kita dapat mencapai ini dengan terlebih dahulu memfilter baris yang berisi judul film input dan kemudian menggabungkan subset ini dengan input dataframe. Kami juga menjatuhkan kolom yang tidak perlu untuk input untuk menghemat ruang memori.

In [0]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


Kita akan mulai dengan mempelajari preferensi input, jadi mari kita ambil subset film yang inputnya tonton dari Dataframe yang mengandung genre yang didefinisikan dengan nilai-nilai biner.

In [0]:
#Menyaring film dari input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Kita hanya perlu tabel genre aktual, jadi mari kita bersihkan ini sedikit.

##### **Kegiatan 5 : 10 Poin**
Atur ulang indeks drop kolom film Id, judul, genre dan kolom tahun.

In [0]:
# TULIS KODE ANDA DISINI

"""CONTOH OUTPUT"""

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sekarang kita mulai mempelajari preferensi input!

Untuk melakukan ini, kita akan mengubah setiap genre menjadi bobot. Kita dapat melakukan ini dengan menggunakan input ulasan dan mengalikannya ke dalam tabel genre input dan kemudian merangkum tabel yang dihasilkan oleh kolom. Operasi ini sebenarnya merupakan produk titik antara matriks dan vektor, jadi kita bisa melakukannya dengan memanggil fungsi "dot" Pandas.

In [0]:
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [0]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Sekarang, kita memiliki bobot untuk setiap preferensi pengguna. Ini dikenal sebagai Profil Pengguna. Dengan menggunakan ini, kami dapat merekomendasikan film yang memenuhi preferensi pengguna.

Mari kita mulai dengan mengekstraksi tabel genre dari kerangka data asli:

In [0]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
genreTable.shape

##### **Kegiatan 6 : 30 Poin**

Dengan profil input dan daftar lengkap film serta genre mereka, Kita akan mengambil rata-rata tertimbang dari setiap film berdasarkan profil input dan merekomendasikan dua puluh film teratas yang paling memuaskan.

In [0]:
# LENGKAPI 2 KODE DIBAWAH
"""1. Multiply the genres by the weights and then take the weighted average"""
recommendationTable_df = .................

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

In [0]:
"""2. Sort our recommendations in descending order"""
recommendationTable_df = ............................
# Tampilkan isi dataset
recommendationTable_df.head()

movieId
5018      0.748252
26093     0.734266
27344     0.720280
148775    0.685315
6902      0.678322
dtype: float64

## **Sekarang inilah tabel rekomendasi!**

##### **Kegiatan 7 : 20 Poin**

Buat tabel rekomendasi akhir.

In [0]:
#The final recommendation table

"""HANYA CONTOH!!"""

,movieId,title,genres,year
664,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1824,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2902,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
4923,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6793,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
8605,26093,"Wonderful World of the Brothers Grimm, The","[Adventure, Animation, Children, Comedy, Drama...",1962
8783,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
9825,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
11716,51632,Atlantis: Milo's Return,"[Action, Adventure, Animation, Children, Comed...",2003
